In [3]:
from tqdm import tqdm
from lxml import etree
import os

In [4]:
topic_tree = etree.parse('data/topics2016.xml')
def get_topic(i):# returns the summary string of the ith topic
    summary = topic_tree.xpath('//topic[@number="%d"]/summary/text()'%i)
    return str(summary).lower()

# get_topic(1)

In [5]:
# build a mapping of article name (PMCID) to its file path

PMC_PATH = '/local/XW/DATA/TREC/PMCs/'
pmcid2fpath = {}

for subdir1 in os.listdir(PMC_PATH):
    for subdir2 in os.listdir(os.path.join(PMC_PATH, subdir1)):
        diry = os.path.join(PMC_PATH, subdir1, subdir2)
#         print diry, len(os.listdir(diry))
        for fn in os.listdir(diry):
            pmcid = fn[:-5]
            fpath = os.path.join(diry, fn)
            pmcid2fpath[pmcid] = fpath

# pmcid2fpath['3539039']

In [6]:
def get_article_abstract(pmcid):
    fpath = pmcid2fpath[pmcid]
    tree = etree.parse(fpath)
    abstract = tree.xpath('//abstract')[0]
    ret = u''+abstract.xpath('string(.)')
    return ret.lower()

In [ ]:
get_article_abstract('3535727')

In [ ]:
get_article_abstract('64830')

In [ ]:
print get_article_abstract('4798933')

In [ ]:
print get_article_abstract('107838')

In [ ]:
topicidx = 1

In [69]:
corpus = []
topicid_pmcid_2relevance = {}
with open('data/qrels.txt') as f:
    for line in tqdm(f, total=37707): 
        topicid, _, pmcid, relevance = line.split()
        if int(topicid)>5: break
        try:
            corpus.append(get_article_abstract(pmcid))
            topicid_pmcid_2relevance[(topicid, pmcid)] = int(relevance)
        except: pass

 19%|█▉        | 7138/37707 [00:08<00:39, 773.81it/s]


In [70]:
len(topicid_pmcid_2relevance)

6587

In [60]:
len(corpus)

3765

In [37]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+'0123456789')
tokenizer.fit_on_texts(corpus)

In [38]:
seqs = tokenizer.texts_to_sequences(corpus) # turn texts into seq of ids
# seqs is list of list of integers

In [39]:
word2idx = tokenizer.word_index # dictionary mapping words (str) ---> their index (int)

In [40]:
idx2word = {v:k for k,v in word2idx.iteritems()}

In [44]:
tokenizer.document_count

3765

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(corpus)

def idf(wd):
    if wd not in vectorizer.vocabulary_: return 0.0
    return vectorizer.idf_[ vectorizer.vocabulary_[wd] ]

In [72]:
X.shape

(6587, 34784)

In [81]:
print idf('heart')
print idf('head')
print idf('the')
print idf('in')

2.97298872662
4.37247009203
1.0256759435
1.06668614034


In [80]:
import nltk
nltk.word_tokenize(corpus[0])

[u'backgroundthis',
 u'survey',
 u"'s",
 u'objective',
 u'was',
 u'to',
 u'provide',
 u'planning',
 u'information',
 u'by',
 u'examining',
 u'utilization',
 u'patterns',
 u',',
 u'health',
 u'outcomes',
 u'and',
 u'costs',
 u'associated',
 u'with',
 u'existing',
 u'practices',
 u'in',
 u'the',
 u'management',
 u'of',
 u'postpartum',
 u'women',
 u'and',
 u'their',
 u'infants',
 u'.',
 u'in',
 u'particular',
 u',',
 u'this',
 u'paper',
 u'looks',
 u'at',
 u'a',
 u'subgroup',
 u'of',
 u'women',
 u'who',
 u'score',
 u'\u2265',
 u'12',
 u'on',
 u'the',
 u'edinburgh',
 u'postnatal',
 u'depression',
 u'survey',
 u'(',
 u'epds',
 u')',
 u'.methodsthe',
 u'design',
 u'is',
 u'cross-sectional',
 u'with',
 u'follow-up',
 u'at',
 u'four',
 u'weeks',
 u'after',
 u'postpartum',
 u'hospital',
 u'discharge',
 u'.',
 u'five',
 u'ontario',
 u'hospitals',
 u',',
 u'chosen',
 u'for',
 u'their',
 u'varied',
 u'size',
 u',',
 u'practice',
 u'characteristics',
 u',',
 u'and',
 u'geographic',
 u'location',
 u

In [77]:
nltk.download_gui()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
